In [ ]:
import pandas as pd
from IPython.display import Markdown
import arviz as az
import pymc as pm
from estival.wrappers import pymc as epm
from estival.sampling import tools as esamp

from summer2 import CompartmentalModel
from summer2.parameters import Parameter, DerivedOutput
from estival.model import BayesianCompartmentalModel
import estival.priors as esp
import estival.targets as est

In [ ]:
# Base model construction
model = CompartmentalModel(
    times=[1850.0, 2024.0],
    compartments=["susceptible", "early latent", "late latent", "infectious", "recovered"],
    infectious_compartments=["infectious"],
)
model.set_initial_population({"susceptible": 1000.0, "infectious": 1.0})

We built a model of TB transmission. [@trauer2014]

In [ ]:
# Add latency structures
latency_flows = [
    ["stabilisation", "early latent", "late latent"],
    ["early activation", "early latent", "infectious"],
    ["late activation", "late latent", "infectious"],
]
for flow, source, dest in latency_flows:
    model.add_transition_flow(flow, Parameter(f"{flow} rate"), source, dest)

In [ ]:
# Add infection processes
infection_origins = ["susceptible", "late latent", "recovered"]
for origin in infection_origins:
    model.add_infection_frequency_flow(f"infection_from_{origin}", Parameter("contact rate"), origin, "early latent")

In [ ]:
# Natural history flows
model.add_transition_flow("recovery", Parameter("recovery rate"), "infectious", "recovered")
model.add_death_flow("TB death", Parameter("death rate"), "infectious")

In [ ]:
# Demographic flows
model.add_replacement_birth_flow("birth", "susceptible")

In [ ]:
# Track relevant outputs
for timing in ["early", "late"]:
    model.request_output_for_flow(f"{timing} activation", f"{timing} activation", save_results=False)
inc = model.request_function_output("incidence", DerivedOutput("early activation") + DerivedOutput("late activation"), save_results=False)
model.request_function_output("notifications", inc * Parameter("case detection rate"));

In [ ]:
# Parameters
params = {
    "contact rate": 3.0,
    "stabilisation rate": 1.0,
    "early activation rate": 1.0,
    "late activation rate": 1.0,
    "case detection rate": 1.0,
    "recovery rate": 1.0,
    "death rate": 0.1,
}
param_units = {
    "contact rate": "per day",
    "stabilisation rate": "per day",
    "early activation rate": "per day",
    "late activation rate": "per day",
    "case detection rate": "",
    "recovery rate": "per day",
    "death rate": "per day",
}
fixed_param_table = pd.DataFrame({"values": params, "units": param_units})

In [ ]:
Markdown(fixed_param_table.to_markdown())

In [ ]:
priors = [esp.UniformPrior("contact rate", (1.0, 2.0))]
dummy_target = pd.Series({1900.0: 50.0, 1910.0: 70.0})
targets = [est.NormalTarget("notifications", dummy_target, 20.0)]
bcm = BayesianCompartmentalModel(model, params, priors, targets)

In [ ]:
with pm.Model() as pm_model:
    variables = epm.use_model(bcm)
    idata = pm.sample(step=[pm.DEMetropolisZ(variables)], draws=1000, tune=1000, cores=8, chains=8, idata_kwargs={"log_likelihood": True})

In [ ]:
like_df = esamp.likelihood_extras_for_idata(idata, bcm)
chain, draw = like_df["logposterior"].idxmax()
calib_result = {k.name: float(idata.posterior.isel(chain=chain, draw=draw)[k.name]) for k in priors}

In [ ]:
model.run(params | calib_result)

In [ ]:
az.plot_trace(idata);

In [ ]:
plot = model.get_derived_outputs_df().plot()
plot.scatter(dummy_target.index, dummy_target)

In [ ]:
model.get_outputs_df().plot()